In [6]:
import requests
import pandas as pd
from dotenv import load_dotenv
import time
import os
import plotly.express as px

In [7]:
load_dotenv()
flipside_api_key = os.getenv("1a9af818-0d96-4c9b-b615-8d446ab74d1f") # or can use api key directly

In [8]:
def flipside_api_results(query, api_key, attempts=10, delay=30):
    """
    Creates and retrieves results for a query using Flipside's JSON-RPC API.

    Parameters:
    - query: str, the SQL query to execute.
    - api_key: str, your Flipside API key.
    - attempts: int, number of attempts to poll for query completion.
    - delay: int, delay (in seconds) between polling attempts.

    Returns:
    - pd.DataFrame: DataFrame containing the query results.
    """

    # Step 1: Create the query
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_key
    }
    payload = {
        "jsonrpc": "2.0",
        "method": "createQueryRun",
        "params": [
            {
                "resultTTLHours": 1,
                "maxAgeMinutes": 0,
                "sql": query,
                "tags": {"source": "python-script", "env": "production"},
                "dataSource": "snowflake-default",
                "dataProvider": "flipside"
            }
        ],
        "id": 1
    }

    response = requests.post(url, headers=headers, json=payload)
    response_data = response.json()

    if 'error' in response_data:
        raise Exception(f"Error creating query: {response_data['error']['message']}")

    query_run_id = response_data.get('result', {}).get('queryRun', {}).get('id')
    if not query_run_id:
        raise KeyError(f"Query creation failed. Response: {response_data}")

    # Step 2: Poll for query completion
    for attempt in range(attempts):
        status_payload = {
            "jsonrpc": "2.0",
            "method": "getQueryRunResults",
            "params": [
                {
                    "queryRunId": query_run_id,
                    "format": "json",
                    "page": {"number": 1, "size": 10000}
                }
            ],
            "id": 1
        }
        response = requests.post(url, headers=headers, json=status_payload)
        resp_json = response.json()

        if 'result' in resp_json and 'rows' in resp_json['result']:
            # Convert the rows to a DataFrame
            return pd.DataFrame(resp_json['result']['rows'])

        if 'error' in resp_json and 'not yet completed' in resp_json['error'].get('message', '').lower():
            time.sleep(delay)  # Wait before retrying
        else:
            raise Exception(f"Unexpected error while fetching query results: {resp_json}")

    raise TimeoutError(f"Query did not complete after {attempts} attempts.")

In [9]:
query = """

WITH addresses as (
  SELECT
    column1 AS token_address
  FROM
    (
      VALUES
        (
          LOWER('0x83F20F44975D03b1b09e64809B757c47f942BEeA')
        ),
        --sdai
        (
          LOWER('0x9D39A5DE30e57443BfF2A8307A4256c8797A3497')
        ),
        --susde
        (
          LOWER('0x5d3a536e4d6dbd6114cc1ead35777bab948e3643')
        ) --cdai
    ) AS tokens(column1)
),
assets AS (
  SELECT
    hour as dt,
    SYMBOL,
    price
  FROM
    ethereum.price.ez_prices_hourly
  where
    token_address in (
      select
        token_address
      from
        addresses
    )
    and hour > date('2024-03-19')
),
asset_returns AS (
  SELECT
    dt,
    SYMBOL,
    price,
    LAG(price) OVER (
      PARTITION BY SYMBOL
      ORDER BY
        dt
    ) AS prev_price,
    (
      price - LAG(price) OVER (
        PARTITION BY SYMBOL
        ORDER BY
          dt
      )
    ) / LAG(price) OVER (
      PARTITION BY SYMBOL
      ORDER BY
        dt
    ) AS hourly_return,
  FROM
    assets
),
cumulative_returns AS (
  SELECT
    DT,
    symbol,
    hourly_return,
    (1 + hourly_return) AS hourly_factor,
    EXP(
      SUM(LN(1 + hourly_return)) OVER (
        PARTITION BY symbol
        ORDER BY
          DT ASC
      )
    ) - 1 AS cumulative_return
  FROM
    asset_returns
),
base_return AS (
  SELECT
    dt,
    symbol,
    cumulative_return AS base_cumulative_return
  FROM
    cumulative_returns
  WHERE
    cumulative_return IS NOT NULL
),
normalized_returns AS (
  SELECT
    dt,
    symbol,
    100 + (
      100 * (
        base_cumulative_return - FIRST_VALUE(base_cumulative_return) OVER (
          Partition by Symbol
          ORDER BY
            dt ASC
        )
      )
    ) AS norm_asset_return
  FROM
    base_return
)
SELECT
  *
FROM
  normalized_returns
ORDER BY
  dt DESC;

"""

In [10]:
df = flipside_api_results(query,flipside_api_key)

KeyError: 'message'

In [ ]:
df.set_index('dt',inplace=True)
df.index = pd.to_datetime(df.index)
df.drop(columns='__row_index',inplace=True)

In [ ]:
df

fig = px.line(
    df,
    x=df.index,
    y='norm_asset_return',
    line_group='symbol',
    color='symbol'
)

NameError: name 'df' is not defined

In [ ]:
fig.show()